In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from keras.layers import add, Conv2D, Input
from keras.models import Model, load_model
from keras import backend as K
import os
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
def noise(img):
  svp = 0.5 
  amt = 1 
  out = np.copy(img)
  #Salt
  ns = np.ceil(amt*img.size * svp)
  pix = [np.random.randint(0,i,int(ns)) for i in img.shape] 
  out[pix] = 1
  #Pepper
  npp = np.ceil(amt*img.size * (1. - svp))
  pix = [np.random.randint(0,i,int(npp)) for i in img.shape] 
  out[pix] = 0
  return out

In [3]:
def img_prep(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) 
    hsv[...,2] = hsv[...,2]*0.2
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    img = noise(img)
    return img

In [13]:
path= "/home/rohit/Projects/Datasets"

In [22]:
os.chdir("/home/rohit/Projects/Datasets/images")

In [31]:
ab1= np.load('ab1.npy', fix_imports=True)
ab2= np.load('ab2.npy', fix_imports=True)
ab3= np.load('ab3.npy', fix_imports=True)

In [32]:
ab= np.concatenate((ab1,ab2,ab3), axis=0)

In [34]:
l= np.load('gray_scale.npy', fix_imports=True)

In [48]:
l= l.reshape(25000,224,224,1)

In [51]:
X_ = l[0:10000,...]

In [52]:
merged_images= cv2.merge(X_, ab1)

error: OpenCV(4.0.0) /io/opencv/modules/core/src/merge.cpp:300: error: (-215:Assertion failed) 0 < cn && cn <= CV_CN_MAX in function 'merge'


In [ ]:
images = cv2.cvtColor(merged_images, cv2.COLOR_LAB2BGR)

In [21]:
X = []
y = []
l=0
try:
for i in tqdm(os.listdir(path)):
    img = cv2.imread(os.path.join(path,i))
    img = cv2.resize(img, (224,224))
    y.append(img)
    X.append(img_prep(img))
    l = l+1
    if l>1500:
        break
X=np.array(X)
y=np.array(y)

  0%|          | 0/2 [00:00<?, ?it/s]


error: OpenCV(4.0.0) /io/opencv/modules/imgproc/src/resize.cpp:3784: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
K.clear_session()
def InstantiateModel(in_):
    
    model_1 = Conv2D(16,(3,3), activation='relu',padding='same',strides=1)(in_)
    model_1 = Conv2D(32,(3,3), activation='relu',padding='same',strides=1)(model_1)
    model_1 = Conv2D(64,(2,2), activation='relu',padding='same',strides=1)(model_1)
    
    model_2 = Conv2D(32,(3,3), activation='relu',padding='same',strides=1)(in_)
    model_2 = Conv2D(64,(2,2), activation='relu',padding='same',strides=1)(model_2)
    
    model_2_0 = Conv2D(64,(2,2), activation='relu',padding='same',strides=1)(model_2)
    
    model_add = add([model_1,model_2,model_2_0])
    
    model_3 = Conv2D(64,(3,3), activation='relu',padding='same',strides=1)(model_add)
    model_3 = Conv2D(32,(3,3), activation='relu',padding='same',strides=1)(model_3)
    model_3 = Conv2D(16,(2,2), activation='relu',padding='same',strides=1)(model_3)
    
    model_3_1 = Conv2D(32,(3,3), activation='relu',padding='same',strides=1)(model_add)
    model_3_1 = Conv2D(16,(2,2), activation='relu',padding='same',strides=1)(model_3_1)
    
    model_3_2 = Conv2D(16,(2,2), activation='relu',padding='same',strides=1)(model_add)
    
    model_add_2 = add([model_3_1,model_3_2,model_3])
    
    model_4 = Conv2D(16,(3,3), activation='relu',padding='same',strides=1)(model_add_2)
    model_4_1 = Conv2D(16,(3,3), activation='relu',padding='same',strides=1)(model_add)
    #Extension
    model_add_3 = add([model_4_1,model_add_2,model_4])
    
    model_5 = Conv2D(16,(3,3), activation='relu',padding='same',strides=1)(model_add_3)
    model_5 = Conv2D(16,(2,2), activation='relu',padding='same',strides=1)(model_add_3)
    
    model_5 = Conv2D(3,(3,3), activation='relu',padding='same',strides=1)(model_5)
    
    return model_5

In [ ]:
Input_Sample = Input(shape=(224, 224,3))
Output_ = InstantiateModel(Input_Sample)
model = Model(inputs=Input_Sample, outputs=Output_)

In [ ]:
model.compile(optimizer="adam", loss='mean_squared_error', metrics = ['accuracy'])

In [ ]:
def GI(X,y):
    for i in range(len(X)):
        X_ = X[i].reshape(1,224,224,3)
        y_ = y[i].reshape(1,224,224,3)
        yield (X_,y_)

In [ ]:
Training = False

In [ ]:
if Training:
    model.fit_generator(GI(X_,y),epochs=50,verbose=1,steps_per_epoch=100,shuffle=True)
    model.save("Low_light_enhancement_v1_224_X_224_1.h5")
else:
    model = load_model("Low_light_enhancement_v1_224_X_224.h5")